# Ember-a-hurokban munkafolyamat a Microsoft Agent Framework segítségével

## 🎯 Tanulási célok

Ebben a jegyzetfüzetben megtanulhatod, hogyan valósíts meg **ember-a-hurokban** munkafolyamatokat a Microsoft Agent Framework `RequestInfoExecutor` használatával. Ez az erőteljes minta lehetővé teszi, hogy az AI munkafolyamatok szüneteljenek emberi input begyűjtésére, interaktívvá téve az ügynököket, és biztosítva az emberek számára a kritikus döntések feletti kontrollt.

## 🔄 Mi az ember-a-hurokban?

Az **ember-a-hurokban (HITL)** egy tervezési minta, amelyben az AI ügynökök szüneteltetik a végrehajtást, hogy emberi inputot kérjenek, mielőtt folytatnák. Ez elengedhetetlen a következő esetekben:

- ✅ **Kritikus döntések** - Emberi jóváhagyás kérése fontos lépések előtt
- ✅ **Kétértelmű helyzetek** - Emberek bevonása, amikor az AI bizonytalan
- ✅ **Felhasználói preferenciák** - Felhasználók választása több opció közül
- ✅ **Megfelelőség és biztonság** - Emberi felügyelet biztosítása szabályozott műveleteknél
- ✅ **Interaktív élmények** - Olyan beszélgető ügynökök létrehozása, amelyek reagálnak a felhasználói inputra

## 🏗️ Hogyan működik a Microsoft Agent Framework-ben?

A keretrendszer három kulcsfontosságú komponenst biztosít az HITL-hez:

1. **`RequestInfoExecutor`** - Egy speciális végrehajtó, amely szünetelteti a munkafolyamatot és kibocsát egy `RequestInfoEvent` eseményt
2. **`RequestInfoMessage`** - Alaposztály a gép által küldött, emberi inputot kérő üzenetekhez
3. **`RequestResponse`** - Emberi válaszok összekapcsolása az eredeti kérésekkel `request_id` használatával

**Munkafolyamat minta:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Példánk: Hotel foglalás felhasználói megerősítéssel

A feltételes munkafolyamatot kibővítjük emberi megerősítéssel, **mielőtt** alternatív célállomásokat javasolnánk:

1. A felhasználó megad egy célállomást (pl. "Párizs")
2. Az `availability_agent` ellenőrzi, hogy vannak-e szabad szobák
3. **Ha nincs szoba** → A `confirmation_agent` megkérdezi: "Szeretnél alternatívákat látni?"
4. A munkafolyamat **szünetel** a `RequestInfoExecutor` használatával
5. **Az ember válaszol** "igen" vagy "nem" konzol bemeneten keresztül
6. A `decision_manager` a válasz alapján irányít:
   - **Igen** → Alternatív célállomások megjelenítése
   - **Nem** → Foglalási kérés törlése
7. Végső eredmény megjelenítése

Ez bemutatja, hogyan adhatunk kontrollt a felhasználóknak az ügynök javaslatai felett!

---

Kezdjük el! 🚀


## 1. lépés: Szükséges könyvtárak importálása

Importáljuk a standard Agent Framework komponenseket, valamint a **human-in-the-loop specifikus osztályokat**:
- `RequestInfoExecutor` - Végrehajtó, amely szünetelteti a munkafolyamatot emberi beavatkozás érdekében
- `RequestInfoEvent` - Esemény, amely akkor kerül kibocsátásra, amikor emberi beavatkozás szükséges
- `RequestInfoMessage` - Alaposztály a típusos kérés-payloadokhoz
- `RequestResponse` - Összekapcsolja az emberi válaszokat a kérésekkel
- `WorkflowOutputEvent` - Esemény a munkafolyamat kimeneteinek észlelésére


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## 2. lépés: Pydantic modellek meghatározása strukturált kimenetekhez

Ezek a modellek határozzák meg az **sémát**, amelyet az ügynökök visszaadnak. Megtartjuk az összes modellt a feltételes munkafolyamatból, és hozzáadjuk:

**Új az emberi beavatkozáshoz:**
- `HumanFeedbackRequest` - A `RequestInfoMessage` alosztálya, amely meghatározza az embereknek küldött kérés adatait
  - Tartalmazza a `prompt` (feltenni kívánt kérdés) és a `destination` (kontekstus az elérhetetlen városról) elemeket


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## 3. lépés: Hozza létre a szállásfoglalási eszközt

Ugyanaz az eszköz, mint a feltételes munkafolyamatnál - ellenőrzi, hogy vannak-e szabad szobák a célállomáson.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## 4. lépés: Feltétel függvények meghatározása az útvonalhoz

Négy **feltétel függvényre** van szükségünk az emberi beavatkozást igénylő munkafolyamathoz:

**A feltételes munkafolyamatból:**
1. `has_availability_condition` - Útvonal, amikor vannak elérhető szállodák
2. `no_availability_condition` - Útvonal, amikor nincsenek elérhető szállodák

**Új az emberi beavatkozáshoz:**
3. `user_wants_alternatives_condition` - Útvonal, amikor a felhasználó igent mond az alternatívákra
4. `user_declines_alternatives_condition` - Útvonal, amikor a felhasználó nemet mond az alternatívákra


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## 5. lépés: Hozza létre a Decision Manager Executor-t

Ez a **human-in-the-loop minta lényege**! A `DecisionManager` egy egyedi `Executor`, amely:

1. **Fogadja az emberi visszajelzést** `RequestResponse` objektumokon keresztül
2. **Feldolgozza a felhasználó döntését** (igen/nem)
3. **Irányítja a munkafolyamatot** azáltal, hogy üzeneteket küld a megfelelő ügynököknek

Főbb jellemzők:
- Az `@handler` dekorátor segítségével teszi elérhetővé a metódusokat munkafolyamat lépésként
- `RequestResponse[HumanFeedbackRequest, str]`-t kap, amely tartalmazza az eredeti kérést és a felhasználó válaszát
- Egyszerű "igen" vagy "nem" üzeneteket ad vissza, amelyek aktiválják a feltétel funkcióinkat


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## 6. lépés: Egyéni megjelenítési végrehajtó létrehozása

Ugyanaz a megjelenítési végrehajtó, mint a feltételes munkafolyamatban - végső eredményeket ad vissza a munkafolyamat kimeneteként.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## 7. lépés: Környezeti változók betöltése

Konfigurálja az LLM klienst (GitHub Models, Azure OpenAI vagy OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## 8. lépés: AI ügynökök és végrehajtók létrehozása

Hat **munkafolyamat-komponenst** hozunk létre:

**Ügynökök (AgentExecutor-ba csomagolva):**
1. **availability_agent** - Ellenőrzi a szállodai elérhetőséget az eszköz segítségével
2. **confirmation_agent** - 🆕 Előkészíti az emberi megerősítési kérést
3. **alternative_agent** - Alternatív városokat javasol (amikor a felhasználó igent mond)
4. **booking_agent** - Foglalásra ösztönöz (amikor vannak szabad szobák)
5. **cancellation_agent** - 🆕 Kezeli a lemondási üzenetet (amikor a felhasználó nemet mond)

**Speciális végrehajtók:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, amely szünetelteti a munkafolyamatot emberi beavatkozásra
7. **decision_manager** - 🆕 Egyedi végrehajtó, amely az emberi válasz alapján irányít (már korábban definiálva)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## 9. lépés: Munkafolyamat létrehozása emberi beavatkozással

Most felépítjük a munkafolyamat gráfját **feltételes útvonalvezetéssel**, beleértve az emberi beavatkozás útját is:

**Munkafolyamat szerkezete:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Kulcsfontosságú élek:**
- `availability_agent → confirmation_agent` (ha nincs szabad szoba)
- `confirmation_agent → prepare_human_request` (típus átalakítása)
- `prepare_human_request → request_info_executor` (szünet emberi beavatkozásra)
- `request_info_executor → decision_manager` (mindig - RequestResponse-t biztosít)
- `decision_manager → alternative_agent` (ha a felhasználó "igen"-t mond)
- `decision_manager → cancellation_agent` (ha a felhasználó "nem"-et mond)
- `availability_agent → booking_agent` (ha van szabad szoba)
- Minden útvonal a `display_result`-nál ér véget


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## 10. lépés: Teszteset 1 futtatása - Város ELÉRHETŐSÉG NÉLKÜL (Párizs emberi megerősítéssel)

Ez a teszt bemutatja a **teljes emberi beavatkozási ciklust**:

1. Szálloda kérése Párizsban
2. availability_agent ellenőriz → Nincs szoba
3. confirmation_agent emberi kérdést hoz létre
4. request_info_executor **szünetelteti a munkafolyamatot** és kibocsát egy `RequestInfoEvent` eseményt
5. **Az alkalmazás érzékeli az eseményt és konzolon kéri a felhasználót**
6. A felhasználó beírja, hogy "igen" vagy "nem"
7. Az alkalmazás válaszokat küld a `send_responses_streaming()` segítségével
8. decision_manager a válasz alapján irányít
9. A végső eredmény megjelenik

**Kulcsminta:**
- Használja a `workflow.run_stream()`-et az első iterációhoz
- Használja a `workflow.send_responses_streaming(pending_responses)`-t a következő iterációkhoz
- Figyelje a `RequestInfoEvent`-et, hogy érzékelje, mikor van szükség emberi beavatkozásra
- Figyelje a `WorkflowOutputEvent`-et, hogy rögzítse a végső eredményeket


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## 11. lépés: Futtassa a 2. tesztesetet - Város ELÉRHETŐSÉGGEL (Stockholm - Nincs szükség emberi beavatkozásra)

Ez a teszt bemutatja a **közvetlen utat**, amikor szobák elérhetők:

1. Szálloda kérése Stockholmban
2. availability_agent ellenőriz → Szobák elérhetők ✅
3. booking_agent javasolja a foglalást
4. display_result megjeleníti a visszaigazolást
5. **Nincs szükség emberi beavatkozásra!**

A munkafolyamat teljesen megkerüli az emberi beavatkozást, amikor szobák elérhetők.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Főbb tanulságok és emberi beavatkozásra épülő legjobb gyakorlatok

### ✅ Amit megtanultál:

#### 1. **RequestInfoExecutor minta**
A Microsoft Agent Framework emberi beavatkozásra épülő mintája három kulcskomponenst használ:
- `RequestInfoExecutor` - Megállítja a munkafolyamatot és eseményeket bocsát ki
- `RequestInfoMessage` - Típusos kérési adatok alap osztálya (származtass ebből!)
- `RequestResponse` - Összekapcsolja az emberi válaszokat az eredeti kérésekkel

**Fontos megértés:**
- A `RequestInfoExecutor` NEM gyűjti be az adatokat - csak megállítja a munkafolyamatot
- Az alkalmazásod kódjának figyelnie kell a `RequestInfoEvent` eseményre és be kell gyűjtenie az adatokat
- A `send_responses_streaming()` függvényt kell meghívnod egy szótárral, amely a `request_id`-t a felhasználó válaszával párosítja

#### 2. **Streaming végrehajtási minta**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Eseményvezérelt architektúra**
Figyelj specifikus eseményekre a munkafolyamat irányításához:
- `RequestInfoEvent` - Emberi beavatkozás szükséges (munkafolyamat szünetel)
- `WorkflowOutputEvent` - Végső eredmény elérhető (munkafolyamat befejeződött)
- `WorkflowStatusEvent` - Állapotváltozások (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, stb.)

#### 4. **Egyedi végrehajtók @handler-rel**
A `DecisionManager` bemutatja, hogyan hozhatsz létre végrehajtókat, amelyek:
- Az `@handler` dekorátort használják a metódusok munkafolyamat lépésként való kitettségéhez
- Típusos üzeneteket fogadnak (pl. `RequestResponse[HumanFeedbackRequest, str]`)
- Üzeneteket küldenek más végrehajtóknak a munkafolyamat irányításához
- Hozzáférnek a kontextushoz a `WorkflowContext` segítségével

#### 5. **Feltételes irányítás emberi döntésekkel**
Létrehozhatsz feltétel függvényeket, amelyek értékelik az emberi válaszokat:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Valós alkalmazások:

1. **Jóváhagyási munkafolyamatok**
   - Menedzser jóváhagyásának megszerzése költségjelentések feldolgozása előtt
   - Emberi felülvizsgálat szükséges automatikus e-mailek küldése előtt
   - Nagy értékű tranzakciók megerősítése végrehajtás előtt

2. **Tartalom moderálás**
   - Kétes tartalom megjelölése emberi felülvizsgálatra
   - Moderátorok döntése az egyértelműtlen esetekben
   - Emberi beavatkozás kérése, ha az AI biztonsága alacsony

3. **Ügyfélszolgálat**
   - Az AI automatikusan kezeli a rutinkérdéseket
   - Bonyolult problémák továbbítása emberi ügyintézőknek
   - Megkérdezni az ügyfelet, hogy szeretne-e emberrel beszélni

4. **Adatfeldolgozás**
   - Emberek bevonása az egyértelműtlen adatbejegyzések megoldásába
   - Az AI által értelmezett dokumentumok megerősítése
   - Felhasználók döntése több érvényes értelmezés közül

5. **Biztonságkritikus rendszerek**
   - Emberi megerősítés szükséges visszafordíthatatlan műveletek előtt
   - Jóváhagyás kérése érzékeny adatok elérése előtt
   - Döntések megerősítése szabályozott iparágakban (egészségügy, pénzügy)

6. **Interaktív ügynökök**
   - Olyan beszélgető botok létrehozása, amelyek követő kérdéseket tesznek fel
   - Olyan varázslók készítése, amelyek végigvezetik a felhasználókat összetett folyamatokon
   - Olyan ügynökök tervezése, amelyek lépésről lépésre együttműködnek az emberekkel

### 🔄 Összehasonlítás: Emberi beavatkozással és anélkül

| Jellemző | Feltételes munkafolyamat | Emberi beavatkozásra épülő munkafolyamat |
|----------|--------------------------|-----------------------------------------|
| **Végrehajtás** | Egyszeri `workflow.run()` | Ciklus `run_stream()` + `send_responses_streaming()` |
| **Felhasználói bemenet** | Nincs (teljesen automatizált) | Interaktív kérdések `input()` vagy UI segítségével |
| **Komponensek** | Ügynökök + Végrehajtók | + RequestInfoExecutor + DecisionManager |
| **Események** | Csak AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, stb. |
| **Szünetelés** | Nincs szünetelés | Munkafolyamat szünetel a RequestInfoExecutor-nál |
| **Emberi irányítás** | Nincs emberi irányítás | Emberek hoznak kulcsfontosságú döntéseket |
| **Felhasználási terület** | Automatizálás | Együttműködés és felügyelet |

### 🚀 Haladó minták:

#### Több emberi döntési pont
Több `RequestInfoExecutor` csomópontot is használhatsz ugyanabban a munkafolyamatban:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Időtúllépés kezelése
Időtúllépést valósíthatsz meg az emberi válaszokhoz:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Gazdag UI integráció
Az `input()` helyett integrálj webes UI-val, Slackkel, Teams-szel stb.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Feltételes emberi beavatkozás
Csak bizonyos helyzetekben kérj emberi beavatkozást:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Legjobb gyakorlatok:

1. **Mindig származtass a RequestInfoMessage-ből**
   - Típusbiztonságot és validációt biztosít
   - Gazdag kontextust tesz lehetővé UI megjelenítéshez
   - Egyértelművé teszi az egyes kérés típusok szándékát

2. **Használj leíró kérdéseket**
   - Tartalmazzon kontextust arról, hogy mit kérdezel
   - Magyarázd el az egyes választások következményeit
   - Tartsd egyszerűnek és érthetőnek a kérdéseket

3. **Kezeld a váratlan bemeneteket**
   - Validáld a felhasználói válaszokat
   - Adj alapértelmezett értékeket érvénytelen bemenet esetén
   - Nyújts egyértelmű hibaüzeneteket

4. **Kövesd a kérés azonosítókat**
   - Használd a request_id és válaszok közötti kapcsolatot
   - Ne próbáld manuálisan kezelni az állapotot

5. **Tervezd nem blokkolóra**
   - Ne blokkolj szálakat bemenetre várva
   - Használj aszinkron mintákat
   - Támogasd az egyidejű munkafolyamat példányokat

### 📚 Kapcsolódó fogalmak:

- **Agent Middleware** - Ügynök hívások elfogása (előző jegyzetfüzet)
- **Munkafolyamat állapotkezelés** - Munkafolyamat állapotának megőrzése futások között
- **Több ügynök együttműködése** - Emberi beavatkozás kombinálása ügynök csapatokkal
- **Eseményvezérelt architektúrák** - Reaktív rendszerek építése eseményekkel

---

### 🎓 Gratulálunk!

Elsajátítottad az emberi beavatkozásra épülő munkafolyamatokat a Microsoft Agent Framework segítségével! Most már tudod, hogyan:
- ✅ Szüneteltesd a munkafolyamatokat emberi bemenet gyűjtéséhez
- ✅ Használd a RequestInfoExecutor-t és a RequestInfoMessage-t
- ✅ Kezeld a streaming végrehajtást eseményekkel
- ✅ Hozz létre egyedi végrehajtókat @handler-rel
- ✅ Irányítsd a munkafolyamatokat emberi döntések alapján
- ✅ Építs interaktív AI ügynököket, amelyek együttműködnek az emberekkel

**Ez egy kritikus minta megbízható, irányítható AI rendszerek építéséhez!** 🚀



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
